In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"masninjuairia","key":"41567a3baaca60806c281076bdf6742e"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets download -d grassknoted/asl-alphabet


Dataset URL: https://www.kaggle.com/datasets/grassknoted/asl-alphabet
License(s): GPL-2.0
 99% 1.01G/1.03G [00:08<00:00, 248MB/s]
100% 1.03G/1.03G [00:08<00:00, 123MB/s]


In [5]:
!unzip asl-alphabet.zip -d ./asl_dataset


Streaming output truncated to the last 5000 lines.
  inflating: ./asl_dataset/asl_alphabet_train/asl_alphabet_train/nothing/nothing19.jpg  
  inflating: ./asl_dataset/asl_alphabet_train/asl_alphabet_train/nothing/nothing190.jpg  
  inflating: ./asl_dataset/asl_alphabet_train/asl_alphabet_train/nothing/nothing1900.jpg  
  inflating: ./asl_dataset/asl_alphabet_train/asl_alphabet_train/nothing/nothing1901.jpg  
  inflating: ./asl_dataset/asl_alphabet_train/asl_alphabet_train/nothing/nothing1902.jpg  
  inflating: ./asl_dataset/asl_alphabet_train/asl_alphabet_train/nothing/nothing1903.jpg  
  inflating: ./asl_dataset/asl_alphabet_train/asl_alphabet_train/nothing/nothing1904.jpg  
  inflating: ./asl_dataset/asl_alphabet_train/asl_alphabet_train/nothing/nothing1905.jpg  
  inflating: ./asl_dataset/asl_alphabet_train/asl_alphabet_train/nothing/nothing1906.jpg  
  inflating: ./asl_dataset/asl_alphabet_train/asl_alphabet_train/nothing/nothing1907.jpg  
  inflating: ./asl_dataset/asl_alphabet_tr

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ✅ Settings
IMG_SIZE = 224
BATCH_SIZE = 32
DATASET_PATH = "./asl_dataset/asl_alphabet_train/asl_alphabet_train"

# ✅ Data Augmentation and Preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Split 20% for validation
)

# ✅ Training Data Generator
train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# ✅ Validation Data Generator
val_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# ✅ Number of Classes
num_classes = len(train_generator.class_indices)

print(f"✅ Number of classes: {num_classes}")
print(f"✅ Training samples: {train_generator.samples}")
print(f"✅ Validation samples: {val_generator.samples}")


Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.
✅ Number of classes: 29
✅ Training samples: 69600
✅ Validation samples: 17400


In [7]:
# ✅ Convert generators to tf.data.Dataset for small batch handling
train_ds = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32)
    )
).batch(32).shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_generator(
    lambda: val_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32)
    )
).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)


In [8]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import mixed_precision

# ✅ Enable Mixed Precision for T4 GPU Optimization
mixed_precision.set_global_policy('mixed_float16')

# ✅ Settings
IMG_SIZE = 224
BATCH_SIZE = 32
DATASET_PATH = "./asl_dataset/asl_alphabet_train/asl_alphabet_train"


In [9]:
# ✅ Load Dataset and Split into Small Batches
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    label_mode='categorical',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="training",
    seed=42,
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    label_mode='categorical',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=42,
    shuffle=True
)

# ✅ Save class_names BEFORE prefetching!
class_names = train_ds.class_names
num_classes = len(class_names)
print(f"✅ Number of classes: {num_classes}")

# ✅ Now Apply Prefetch
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
# ✅ Adjust Steps per Epoch (2700 total steps)
steps_per_epoch = 2700 // BATCH_SIZE
validation_steps = len(val_ds) // BATCH_SIZE


Found 87000 files belonging to 29 classes.
Using 69600 files for training.
Found 87000 files belonging to 29 classes.
Using 17400 files for validation.
✅ Number of classes: 29


In [10]:
# ✅ Load Pretrained ResNet50 Model (Frozen Base)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(num_classes, activation='softmax')  # ✅ Use saved num_classes
])


# ✅ Compile the Model (Use Mixed Precision)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ✅ Model Summary
model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 29)                  │           7,453 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,119,709 (92.01 MB)

 Trainable params: 531,997 (2.03 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [11]:
# ✅ Early Stopping to Avoid Overfitting
early_stop = EarlyStopping(patience=3, restore_best_weights=True)

# ✅ Train Model
history_cnn = model.fit(train_ds, validation_data=val_ds, epochs=10, steps_per_epoch=None)


Epoch 1/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 129s 51ms/step - accuracy: 0.7853 - loss: 0.7503 - val_accuracy: 0.9866 - val_loss: 0.0469
Epoch 2/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 88s 40ms/step - accuracy: 0.9609 - loss: 0.1153 - val_accuracy: 0.9920 - val_loss: 0.0260
Epoch 3/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 86s 40ms/step - accuracy: 0.9704 - loss: 0.0844 - val_accuracy: 0.9921 - val_loss: 0.0244
Epoch 4/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 87s 40ms/step - accuracy: 0.9728 - loss: 0.0774 - val_accuracy: 0.9935 - val_loss: 0.0179
Epoch 5/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 144s 41ms/step - accuracy: 0.9792 - loss: 0.0638 - val_accuracy: 0.9963 - val_loss: 0.0115
Epoch 6/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 88s 40ms/step - accuracy: 0.9793 - loss: 0.0623 - val_accuracy: 0.9956 - val_loss: 0.0149
Epoch 7/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 84s 38ms/step - accuracy: 0.9827 - loss: 0.0527 - val_accuracy: 0.9952 - val_loss: 0.0135
Epoch 8/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 147s 41ms/step - accuracy: 0.982

In [12]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive

# ✅ Step 1: Define Evaluation Metrics
def compute_metrics(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    acc = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1-score": f1
    }

print("✅ Evaluation Metrics Defined!")

# ✅ Step 2: Define Custom Evaluation Function for ResNet50 Model
class ResNetEvaluator:
    def __init__(self, model, val_ds):
        self.model = model
        self.val_ds = val_ds

    def evaluate(self):
        print("\n🔍 Evaluating ResNet50 Model...")

        y_true = []
        y_pred = []

        for images, labels in self.val_ds:
            logits = self.model.predict(images, verbose=0)
            predictions = np.argmax(logits, axis=-1)

            y_true.extend(np.argmax(labels.numpy(), axis=-1))
            y_pred.extend(predictions)

        y_true = np.array(y_true)
        y_pred = np.array(y_pred)

        # ✅ Compute Metrics
        metrics = compute_metrics(y_true, y_pred)

        print("\n✅ FINAL MODEL RESULTS ✅")
        for key, value in metrics.items():
            print(f"{key}: {value:.5f}")

        return metrics

# ✅ Step 3: Run Evaluation for ResNet50 Model
if 'model' in locals():
    evaluator = ResNetEvaluator(model, val_ds)
    eval_results = evaluator.evaluate()

    # ✅ Save to Google Drive
    drive.mount('/content/drive')
    model.save('/content/drive/MyDrive/asl_resnet50_batched.h5')

    print("✅ ResNet50 model saved to Google Drive!")

✅ Evaluation Metrics Defined!

🔍 Evaluating ResNet50 Model...

✅ FINAL MODEL RESULTS ✅
accuracy: 0.99408
precision: 0.99417
recall: 0.99408
f1-score: 0.99408


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ ResNet50 model saved to Google Drive!


In [13]:
import numpy as np
import matplotlib.pyplot as plt

# ✅ Define evaluation metrics (from previous evaluation results for ResNet50)
eval_metrics = {
    "accuracy": 0.9940,
    "precision": 0.99417,
    "recall": 0.99408,
    "f1-score": 0.99408
}

# ✅ Function to plot training accuracy & loss
def plot_training_history(history, model_name="ResNet50"):
    plt.figure(figsize=(12, 5))

    # ✅ Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history['accuracy'], label='Train Accuracy', marker='o')
    plt.plot(history['val_accuracy'], label='Validation Accuracy', marker='o')
    plt.title(f'{model_name} - Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # ✅ Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(history['loss'], label='Train Loss', marker='o', color='red')
    plt.plot(history['val_loss'], label='Validation Loss', marker='o', color='purple')
    plt.title(f'{model_name} - Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# ✅ Function to plot Accuracy, Precision, Recall, and F1-score side by side
def plot_eval_metrics(metrics):
    plt.figure(figsize=(12, 5))

    metric_names = list(metrics.keys())
    metric_values = list(metrics.values())
    colors = ["blue", "green", "orange", "red"]

    plt.bar(metric_names, metric_values, color=colors, width=0.5)
    plt.ylabel("Score")
    plt.ylim(0.98, 1.0)  # Adjust y-axis for better visualization
    plt.title("Evaluation Metrics (Accuracy, Precision, Recall, F1-score)")

    for i, v in enumerate(metric_values):
        plt.text(i, v, f"{v:.5f}", ha='center', va='bottom', fontsize=12, fontweight='bold')

    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

# ✅ Simulated training history (Replace this with actual history from ResNet50)
history_resnet = {
    'accuracy': np.linspace(0.85, 0.99, 10).tolist(),
    'val_accuracy': np.linspace(0.80, 0.97, 10).tolist(),
    'loss': np.linspace(0.6, 0.1, 10).tolist(),
    'val_loss': np.linspace(0.7, 0.15, 10).tolist()
}

# ✅ Plot training accuracy & loss for ResNet50
plot_training_history(history_resnet, model_name="ResNet50")

# ✅ Plot Accuracy, Precision, Recall & F1-score together for ResNet50
plot_eval_metrics(eval_metrics)